In [ ]:
import sys
import matplotlib

import matplotlib.pyplot as plt
import yaml
import pandas as pd
import numpy as np
import nlopt
from itertools import product
from progressbar import printProgressBar

import pcse
from pcse.models import Wofost72_PP, Wofost72_WLP_FD
from pcse.base import ParameterProvider
from pcse.db import NASAPowerWeatherDataProvider
from pcse.fileinput import YAMLAgroManagementReader, YAMLCropDataProvider
from pcse.util import WOFOST72SiteDataProvider, DummySoilDataProvider
import scipy

In [ ]:
%matplotlib inline
matplotlib.style.use("ggplot")

In [ ]:
# Weather data for Netherlands
wdp = NASAPowerWeatherDataProvider(latitude=52, longitude=5)

# Standard crop parameter library
cropd = YAMLCropDataProvider()

# We don't need soil for potential production, so we use dummy values
soild = DummySoilDataProvider()

# Some site parameters
sited = WOFOST72SiteDataProvider(WAV=50, CO2=360.)

# Package everyting into a single parameter object
params = ParameterProvider(cropdata=cropd, sitedata=sited, soildata=soild)

In [ ]:
agro_yaml = """
- 2006-01-01:
    CropCalendar:
        crop_name: sugarbeet
        variety_name: Sugarbeet_603
        crop_start_date: 2006-03-31
        crop_start_type: emergence
        crop_end_date: 2006-10-20
        crop_end_type: harvest
        max_duration: 300
    TimedEvents:
    -   event_signal: irrigate
        name: Irrigation application table
        comment: All irrigation amounts in cm
        events_table:
        - 2006-07-01: {{amount: {a0}, efficiency: 0.7}}
        - 2006-08-01: {{amount: 10, efficiency: 0.7}}
    StateEvents: null
"""

def objective_function(
    irrigation_volume_in_cm: float,
    parameterprovider=params,
    weatherdataprovider=wdp,
) -> float:
    agromanagement = yaml.safe_load(
        agro_yaml.format(a0=irrigation_volume_in_cm)
    )
    wofost = Wofost72_PP(parameterprovider, weatherdataprovider, agromanagement)
    wofost.run_till_terminate()
    output = wofost.get_output()
    df_output = pd.DataFrame(output)
    return -df_output["LAI"].sum()

In [ ]:
type(objective_function(10))

In [ ]:
scipy.optimize.minimize(
    fun=objective_function,
    x0=5, method = "BFGS"
)